In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [9]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
import folium

# Predicting the earthquake intensity which is given by the user.
test_longitude = float(input("Enter longitude : ")) # test value : -118.0  
test_latitude = float(input("Enter latitude : "))  # test value : 34.0  

# Dataset recommended  by IBM
data = pd.read_csv('/kaggle/input/earthquake-database/database.csv')

df = pd.DataFrame(data)

# Normalize longitude and latitude
df['normalized_longitude'] = (df['Longitude'] - df['Longitude'].mean()) / df['Longitude'].std()
df['normalized_latitude'] = (df['Latitude'] - df['Latitude'].mean()) / df['Latitude'].std()

# Scale earthquake intensity between 0 and 1
df['scaled_intensity'] = (df['Magnitude'] - df['Magnitude'].min()) / (df['Magnitude'].max() - df['Magnitude'].min())

# Create a neural network model for regression
model = Sequential()
model.add(Dense(units=32, activation='relu', input_dim=2))
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model using normalized longitude, latitude, and scaled intensity
model.fit(df[['normalized_longitude', 'normalized_latitude']], df['scaled_intensity'], epochs=10, batch_size=32)
 
# Normalize the test longitude and latitude
normalized_test_longitude = (test_longitude - df['Longitude'].mean()) / df['Longitude'].std()
normalized_test_latitude = (test_latitude - df['Latitude'].mean()) / df['Latitude'].std()

# Predict the earthquake intensity for the test location
predicted_intensity = model.predict(np.array([[normalized_test_longitude, normalized_test_latitude]]))[0][0]

Enter longitude :  -118.0 
Enter latitude :  34.0


Epoch 1/10
732/732 [==============================] - 1s 1ms/step - loss: 0.0329
Epoch 2/10
732/732 [==============================] - 1s 1ms/step - loss: 0.0141
Epoch 3/10
732/732 [==============================] - 1s 1ms/step - loss: 0.0139
Epoch 4/10
732/732 [==============================] - 1s 1ms/step - loss: 0.0138
Epoch 5/10
732/732 [==============================] - 1s 1ms/step - loss: 0.0138
Epoch 6/10
732/732 [==============================] - 1s 1ms/step - loss: 0.0138
Epoch 7/10
732/732 [==============================] - 1s 1ms/step - loss: 0.0138
Epoch 8/10
732/732 [==============================] - 1s 1ms/step - loss: 0.0138
Epoch 9/10
732/732 [==============================] - 1s 1ms/step - loss: 0.0138
Epoch 10/10
1/1 [==============================] - 0s 49ms/step


In [7]:
# Create a base map
map = folium.Map(location=[test_latitude, test_longitude], zoom_start=12)

# Adding a circle around intensity area
folium.CircleMarker(
    location=[test_latitude, test_longitude],
    radius= predicted_intensity * 20,
    color='red',
    fill=True,
    fill_color='red',
    fill_opacity=0.6
).add_to(map)

# Saving the map as intensity_map.html 
map.save('intensity_map.html')